In [3]:
# Dự đoán nhiệt độ ngày mai dựa trên nhiệt độ của 7 ngày trước

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Tạo dữ liệu giả lập (Nhiệt độ ngẫu nhiên)
np.random.seed(42)
data = np.random.uniform(15, 35, 100)  # 100 ngày nhiệt độ (giả lập từ 15°C đến 35°C)

# Tạo dữ liệu đầu vào (7 ngày trước -> ngày tiếp theo)
def create_dataset(data, time_steps=7):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 7
X, y = create_dataset(data, time_steps)
X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape để phù hợp với RNN

# Xây dựng mô hình RNN
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(time_steps, 1)),
    Dense(1)  # Dự đoán 1 giá trị (nhiệt độ ngày mai)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, batch_size=8, verbose=1)

# Dự đoán nhiệt độ ngày mai
test_input = data[-7:]  # 7 ngày gần nhất
test_input = test_input.reshape(1, time_steps, 1)  # Reshape cho phù hợp với mô hình
predicted_temp = model.predict(test_input)
print("Dự đoán nhiệt độ ngày mai:", predicted_temp[0][0])



Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 62.8833
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 46.0575 
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 44.5582 
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.7414  
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.4108 
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 43.4583 
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.2832 
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 47.0765 
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 44.3093
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 33.1654
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.9314 
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.6237 
Epoch 13/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.7731 
Epoch 14/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.5055 
Epoch 15/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 

In [10]:
# Dự đoán xem một câu là tích cực hay tiêu cực.

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# Tải dữ liệu IMDB
vocab_size = 10000
maxlen = 100

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Tiền xử lý dữ liệu
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post')

# Xây dựng mô hình LSTM
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=maxlen),
    LSTM(64, activation='tanh'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(
    X_train, y_train,
    epochs=10,  # Tăng số epochs để cải thiện mô hình
    batch_size=64,
    validation_split=0.2
)

# Đánh giá mô hình
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Xử lý câu mới sử dụng Tokenizer của IMDB
new_texts = ["I hate this movie", "This movie is amazing!"]
word_index = imdb.get_word_index()
new_sequences = [[word_index.get(word, 0) for word in text.lower().split()] for text in new_texts]
new_X = pad_sequences(new_sequences, maxlen=maxlen, padding='post')

# Dự đoán cảm xúc
predictions = model.predict(new_X)
for i, prob in enumerate(predictions.flatten()):
    label = "Tích cực" if prob >= 0.5 else "Tiêu cực"
    print(f"Câu: '{new_texts[i]}' -> {label} ({prob:.2f})")


Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 85ms/step - accuracy: 0.6569 - loss: 0.6024 - val_accuracy: 0.8374 - val_loss: 0.3597
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 90ms/step - accuracy: 0.8810 - loss: 0.3073 - val_accuracy: 0.8426 - val_loss: 0.3817
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 87ms/step - accuracy: 0.9165 - loss: 0.2240 - val_accuracy: 0.8408 - val_loss: 0.3777
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 88ms/step - accuracy: 0.9435 - loss: 0.1620 - val_accuracy: 0.8440 - val_loss: 0.3797
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 88ms/step - accuracy: 0.9606 - loss: 0.1269 - val_accuracy: 0.8438 - val_loss: 0.4856
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.9663 - loss: 0.1056 - val_accuracy: 0.8410 - val_loss: 0.4722
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 87ms/step - accuracy: 0.9768 - loss: 0.0768 - val_accuracy: 0.8402 - val_loss: 0.5739
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.9809 - loss: 0.0647 - 

In [ ]:
# Dịch chuỗi dữ liệu cảm biến IOT thành các sự kiện hoặc thông báo

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Tạo dữ liệu giả lập
num_samples = 1000  # Số lượng mẫu tín hiệu
max_sequence_length = 5  # Độ dài chuỗi tín hiệu
input_dim = 50  # Giá trị tối đa của tín hiệu (ví dụ: nhiệt độ từ 0-50 độ C)
output_vocab_size = 5  # Số lượng thông báo (giải thích tín hiệu)

# Dữ liệu đầu vào (tín hiệu IoT)
X = np.random.randint(0, input_dim, (num_samples, max_sequence_length))

# Chuyển đổi X sang one-hot vector
X_one_hot = np.zeros((num_samples, max_sequence_length, input_dim), dtype="float32")
for i in range(num_samples):
    for t, idx in enumerate(X[i]):
        X_one_hot[i, t, idx] = 1

# Dữ liệu đầu ra (chuỗi thông báo giải thích)
Y = np.random.randint(0, output_vocab_size, (num_samples, max_sequence_length))
Y_one_hot = np.zeros((num_samples, max_sequence_length, output_vocab_size), dtype="float32")
for i in range(num_samples):
    for t, idx in enumerate(Y[i]):
        Y_one_hot[i, t, idx] = 1

# Xây dựng mô hình Encoder-Decoder
# Encoder
encoder_inputs = Input(shape=(max_sequence_length, input_dim))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_sequence_length, output_vocab_size))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Mô hình tổng thể
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Huấn luyện mô hình
history = model.fit(
    [X_one_hot, Y_one_hot], Y_one_hot,
    batch_size=64,
    epochs=20,
    validation_split=0.2
)

# Dự đoán trên chuỗi tín hiệu mới
new_signals = np.random.randint(0, input_dim, (1, max_sequence_length))
new_signals_one_hot = np.zeros((1, max_sequence_length, input_dim), dtype="float32")
for t, idx in enumerate(new_signals[0]):
    new_signals_one_hot[0, t, idx] = 1

new_decoder_input = np.zeros((1, max_sequence_length, output_vocab_size))  # Bắt đầu với đầu vào trống
predicted_messages = model.predict([new_signals_one_hot, new_decoder_input])

# Chuyển từ mã số thông báo sang chuỗi văn bản
predicted_message_indices = np.argmax(predicted_messages, axis=-1)
messages = ["Nhiệt độ bình thường", "Nhiệt độ cao", "Nhiệt độ nguy hiểm", "Nhiệt độ giảm", "Hệ thống ổn định"]
translated_messages = [messages[idx] for idx in predicted_message_indices.flatten()]

print("Tín hiệu cảm biến:", new_signals[0])
print("Thông báo dự đoán:", translated_messages)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.2834 - loss: 1.5929 - val_accuracy: 0.5850 - val_loss: 1.5490
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6531 - loss: 1.5305 - val_accuracy: 0.7570 - val_loss: 1.4836
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7781 - loss: 1.4602 - val_accuracy: 0.8500 - val_loss: 1.3868
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8578 - loss: 1.3203 - val_accuracy: 0.9490 - val_loss: 1.1444
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9631 - loss: 1.0390 - val_accuracy: 0.9690 - val_loss: 0.7984
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9917 - loss: 0.7016 - val_accuracy: 1.0000 - val_loss: 0.5421
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.4792 - val_accuracy: 1.0000 - val_loss: 0.3539
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3161 - val_accuracy: 1.0000 - v